In [0]:
!pip install -q Sastrawi
!pip install -q python-crfsuite
!pip install -q nltk

#import library
import pandas as pd
import numpy as np
import nltk
import Sastrawi
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.tag import CRFTagger
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
import pycrfsuite
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# ngikutin tutorial + buat tester
# butuh all_indo_man_tag_corpus_model, negatif_vania, negative_koto, positif vania, positif koto, stopwords
def normalisasi(tweet):
  normal_tw = tweet.lower() #lowercase
  normal_tw = re.sub('\s+', ' ', normal_tw) # remove extraspace
  normal_tw = normal_tw.strip() #trim depan belakang
  normal_tw = re.sub(r'[^\w\s]','',normal_tw) #buang punctuation, 
  normal_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE)
  #regex huruf yang berulang kaya haiiii (untuk fitur unigram)
  normal_tw = normal_regex.sub(r"\1\1", normal_tw) #buanghuruf yang berulang
  return normal_tw

def remove_stopwords(tweet):
  stopwords = pd.read_csv("stopwords.csv")
  special_list = ['username', 'url', 'sensitive-no']
  token = nltk.word_tokenize(tweet)
  token_afterremoval = []
  for k in token:
    if k not in stopwords and k not in special_list:
      token_afterremoval.append(k)

  str_clean = ' '.join(token_afterremoval)
  return str_clean

def Stemming(tweet):
  token = nltk.word_tokenize(tweet)
  stem_kalimat = []
  for k in token:
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem_kata = stemmer.stem(k.encode('ascii','ignore'))
    stem_kalimat.append(stem_kata)

  stem_kalimat_str = ' '.join(stem_kalimat)
  return stem_kalimat_str


def pra_pemrosesan(list_tweet):
  tweet_clean = []
  for tw in list_tweet:
    normal_tweet = normalisasi(tw)
    nosw_tweet = remove_stopwords(normal_tweet)
#     stem_tweet = Stemming(nosw_tweet)
    tweet_clean.append(nosw_tweet)
  return tweet_clean

def EkstraksiBoW(tweet):
  unigram = CountVectorizer(ngram_range=(1,1), max_features=2000)
  unigram_matrix = unigram.fit_transform(np.array(tweet)).todense()
  nama_fitur = unigram.get_feature_names()
  return unigram_matrix, nama_fitur


def EkstraksiSentimen(list_tweet):
  pos = pd.read_csv("positif_vania.txt", header=None, names=['pos'])
  list_pos = pos['pos'].tolist()
  neg = pd.read_csv("negatif_vania.txt", header=None, names=['neg'])
  list_neg = neg['neg'].tolist()
  
  fitur_sentimen_all = []
  for tweet in list_tweet:
    ##inisiasi value
    emosi = ["positif", "negatif"]
    value = [0,0]
    emosi_value = {}
    for i in range(len(emosi)):
      emosi_value[emosi[i]] = value[i]
    
    list_kata = tweet.split()
  
    for k in list_kata:
      if k in list_pos:
        emosi_value["positif"] += 1
      if k in list_neg:
        emosi_value["negatif"] += 1
    fitur_sentimen_perkalimat = list(emosi_value.values())
    fitur_sentimen_all.append(fitur_sentimen_perkalimat)
  return fitur_sentimen_all


def EkstraksiPOS(list_tweet):
  ct = CRFTagger()
  
  ct.set_model_file("all_indo_man_tag_corpus_model.crf.tagger")
  pos_feat_list = []
  count_tag = []
  for tweet in list_tweet:
    token = nltk.word_tokenize(tweet)
    tag = ct.tag_sents([token])
    flat_tag = [item for sublist in tag for item in sublist]
    pos_count = Counter([j for i,j in flat_tag])
    pos_feat = (pos_count['JJ'], pos_count['NEG'])
    pos_feat_list.append(pos_feat)
  return pos_feat_list


def EkstraksiOrtografi(raw_tweet):
  all_orto_feat = []
  for tw in raw_tweet:
    capital_count = sum(1 for c in tw if c.isupper())
    exclamation_count = sum((1 for c in tw if c == "!"))
    word_len = len(nltk.word_tokenize(tw))
    char_len = len(tw)
    orto_feat = [capital_count, exclamation_count, word_len, char_len]
    all_orto_feat.append(orto_feat)
  return all_orto_feat

def EkstraksiSentimenKoto(list_tweet):
  pos = pd.read_csv("positive_koto.tsv", delimiter="\t", index_col=False, header=None, names=['pos', 'sentiment'])
  list_pos = pos['pos'].tolist()
  list_pos_sent = pos['sentiment'].tolist()
  neg = pd.read_csv("negative_koto.tsv", delimiter="\t", index_col=False, header=None, names=['neg', 'sentiment'])
  list_neg = neg['neg'].tolist()
  list_neg_sent = neg['sentiment'].tolist()
  
  dict_pos = dict(zip(list_pos, list_pos_sent))
  dict_neg = dict(zip(list_neg, list_neg_sent))
  
  fitur_sentimen_all = []
  for tweet in list_tweet:
    ##inisiasi value
    emosi = ["positif", "negatif"]
    value = [0,0]
    emosi_value = {}
    for i in range(len(emosi)):
      emosi_value[emosi[i]] = value[i]
    list_kata = tweet.split()
    list_key_pos = dict_pos.keys()
    list_key_neg = dict_neg.keys()
    for k in list_kata:
      if k in list_key_pos:
        emosi_value["positif"] += dict_pos.get(k)
      if k in list_key_neg:
        emosi_value["negatif"] += -(dict_neg.get(k))
    fitur_sentimen_perkalimat = list(emosi_value.values())
    fitur_sentimen_all.append(fitur_sentimen_perkalimat)
  return fitur_sentimen_all

# buat test
def EkstraksiTfidf(tweet):
  unigram = TfidfVectorizer(ngram_range=(1,1), max_features=6600)
  unigram_matrix = unigram.fit_transform(np.array(tweet)).todense()
  nama_fitur = unigram.get_feature_names()
  return unigram_matrix, nama_fitur

**Baca Data + prepprocessing**

In [0]:
#train
raw_data = pd.read_csv("train_set.csv",
delimiter=",", encoding="Latin-1")
raw_data.head()

raw_tweet = raw_data['tweet']
label = raw_data['sentimen'].tolist()
clean_tweet = pra_pemrosesan(raw_tweet)
clean_tweet[:3]

#tester

raw_data_tester = pd.read_csv("tester_set.csv",
delimiter=",", encoding="Latin-1")
raw_data_tester.head()

raw_tweet_tester = raw_data_tester['tweet']
clean_tweet_tester = pra_pemrosesan(raw_tweet_tester)
clean_tweet_tester[:3]


#test
raw_data_test = pd.read_csv("test_set.csv",
delimiter=",", encoding="Latin-1")
raw_data_test.head()

raw_tweet_test = raw_data_test['tweet']
clean_tweet_test = pra_pemrosesan(raw_tweet_test)
clean_tweet_test[:3]


[u'jadi wanita jangan suka menghancurkan hubungan orang jgn bangga berhasil merusak kebahagian orang silahkan saja tapi ga berkah bahagianya nanti hehe',
 u'sombong apanya kalau sms saja dibls terus',
 u'apadah kamu p cie cie baik kamu cie bebe cie kiwkiw']

**Ekstraksi Fitur**



**1.   Training data**



In [0]:
# cobain untuk tester - untuk clean
unigram_feat_clean, feat_name_clean = EkstraksiBoW(clean_tweet)

sentlex_feat_clean = EkstraksiSentimen(clean_tweet)

postag_feat_clean = EkstraksiPOS(clean_tweet)

orto_feat_clean = EkstraksiOrtografi(clean_tweet)

sentlex_koto_feat_clean = EkstraksiSentimenKoto(clean_tweet)

# cobain untuk test

tfidf_feat_clean, tfidf_feat_name_clean = EkstraksiTfidf(clean_tweet)

In [0]:
# cobain untuk tester - untuk raw
unigram_feat, feat_name = EkstraksiBoW(raw_tweet)

sentlex_feat = EkstraksiSentimen(raw_tweet)

postag_feat = EkstraksiPOS(raw_tweet)

orto_feat = EkstraksiOrtografi(raw_tweet)

sentlex_koto_feat = EkstraksiSentimenKoto(raw_tweet)

# cobain untuk test

tfidf_feat, tfidf_feat_name = EkstraksiTfidf(raw_tweet)


**2.   Tester Data**



In [0]:
# cobain untuk tester - untuk clean
unigram_feat_clean_tester, feat_name_clean_tester = EkstraksiBoW(clean_tweet_tester)

sentlex_feat_clean_tester = EkstraksiSentimen(clean_tweet_tester)

postag_feat_clean_tester = EkstraksiPOS(clean_tweet_tester)

orto_feat_clean_tester = EkstraksiOrtografi(clean_tweet_tester)

sentlex_koto_feat_clean_tester = EkstraksiSentimenKoto(clean_tweet_tester)

# cobain untuk test

tfidf_feat_clean_tester = EkstraksiTfidf(clean_tweet_tester)

In [0]:
# cobain untuk tester - untuk raw
unigram_feat_tester, feat_name_tester = EkstraksiBoW(raw_tweet_tester)

sentlex_feat_tester = EkstraksiSentimen(raw_tweet_tester)

postag_feat_tester = EkstraksiPOS(raw_tweet_tester)

orto_feat_tester = EkstraksiOrtografi(raw_tweet_tester)

sentlex_koto_feat_tester = EkstraksiSentimenKoto(raw_tweet_tester)

# cobain untuk test

tfidf_feat_tester = EkstraksiTfidf(raw_tweet_tester)

**Uji Model**

In [0]:
feat_list = [unigram_feat_clean, sentlex_feat_clean, postag_feat_clean, orto_feat_clean, sentlex_koto_feat_clean]
feat_name = ["Unigram", "Sentimen", "POS", "Ortografi", "Sentimen Koto"]

for f, n in zip(feat_list, feat_name):
  X = f
  y = label
  scoring = ['f1_macro']
  print("LOGISTIC REGRESSION")
  lg = LogisticRegression()
  scores = cross_validate(lg, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  print("DECISION TREE")
  dtc = DecisionTreeClassifier()
  scores = cross_validate(dtc, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")

LOGISTIC REGRESSION
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.8476446145265975)
---------------
DECISION TREE
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.7685186897529117)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Sentimen')
('F1-Score :', 0.7916838705373135)
---------------
DECISION TREE
('Jenis Fitur : ', 'Sentimen')
('F1-Score :', 0.7890882123922349)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'POS')
('F1-Score :', 0.5170408403335769)
---------------
DECISION TREE
('Jenis Fitur : ', 'POS')
('F1-Score :', 0.5065480145412494)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Ortografi')
('F1-Score :', 0.5487824096670157)
---------------
DECISION TREE
('Jenis Fitur : ', 'Ortografi')
('F1-Score :', 0.5311882729703921)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Sentimen Koto')
('F1-Score :', 0.6885957118208752)
---------------
DECISION TREE
('Jenis Fitur : ', 'Sentimen Koto')
('F1-Score :', 0.6395145110692807)
---------------


**Predict - pilih 2 yang paling bagus**

Model yang terpilih : Logistic Regression
1.   Sentlex
2.   Unigram



In [0]:
#1. Sentlex

lg_tester = LogisticRegression()
lg_tester.fit(sentlex_feat, label)

pred_sentlex = lg_tester.predict(sentlex_feat_tester)
pred_sentlex

id_target = raw_data_tester['id']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_sentlex})
df_out.to_csv('submission_sentlex.csv', index=False, header=False) # 

submission = pd.read_csv("submission_sentlex.csv",
delimiter=",", encoding="Latin-1")

submission # hasilnya 79.52%

#2. Unigram
cv = CountVectorizer(ngram_range=(1,1), max_features=6000)
unigram = cv.fit(np.array(clean_tweet))

matrix_train = unigram.transform(clean_tweet)
matrix_tester = unigram.transform(clean_tweet_tester)

lr_unigram = LogisticRegression()
lr_unigram.fit(matrix_train, label)

pred_unigram = lr_unigram.predict(matrix_tester)
print(pred_unigram)

df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_unigram})
df_out.to_csv('submission_unigram.csv', index=False, header=False) # 

submission = pd.read_csv("submission_unigram.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # hasilnya 100%

[1 0 1 1 0 0 0 1 1 0]


,0,1
0,1,0
1,2,1
2,3,1



**3.   Data Test**



Coba tambah fitur TFIDF dan model lain

In [0]:
 feat_list = [unigram_feat_clean, tfidf_feat_clean]
feat_name = ["Unigram", "TFIDF"]

for f, n in zip(feat_list, feat_name):
  X = f
  y = label
  scoring = ['f1_macro']
  print("LOGISTIC REGRESSION")
  lg = LogisticRegression()
  scores = cross_validate(lg, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  print("DECISION TREE")
  dtc = DecisionTreeClassifier()
  scores = cross_validate(dtc, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  print("MULTINOMIAL NAIVE BAYES")
  mnb = MultinomialNB()
  scores = cross_validate(mnb, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  print("SUPER VECTOR MACHINE")
  supervm = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')
  scores = cross_validate(supervm, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")

LOGISTIC REGRESSION
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.8476446145265975)
---------------
DECISION TREE
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.7644667236296823)
---------------
MULTINOMIAL NAIVE BAYES
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.8465681807120788)
---------------
SUPER VECTOR MACHINE
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.8347139289951933)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'TFIDF')
('F1-Score :', 0.8485466308195153)
---------------
DECISION TREE
('Jenis Fitur : ', 'TFIDF')
('F1-Score :', 0.768917334899798)
---------------
MULTINOMIAL NAIVE BAYES
('Jenis Fitur : ', 'TFIDF')
('F1-Score :', 0.8658886074399833)
---------------
SUPER VECTOR MACHINE


**Predict - pilih 3 model yang paling bagus dengan kedua fitur tersebut**


**1. Unigram**



In [0]:
# 1. Logistic Regression
cv = CountVectorizer(ngram_range=(1,1), max_features=6000)
# cv = CountVectorizer(ngram_range=(1,1), max_features=2000)
# cv = CountVectorizer(ngram_range=(1,1), max_features=100000)
unigram = cv.fit(np.array(clean_tweet))

matrix_train = unigram.transform(clean_tweet)
matrix_test = unigram.transform(clean_tweet_test)

lr_unigram = LogisticRegression()
lr_unigram.fit(matrix_train, label)

pred_unigram = lr_unigram.predict(matrix_test)
print(pred_unigram)

id_target = raw_data_test['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_unigram})
df_out.to_csv('submission_unigram_test.csv', index=False, header=False) # 

submission = pd.read_csv("submission_unigram_test.csv",
delimiter=",", encoding="Latin-1")

submission[:3] 

# 2. Multinomial Naive Bayes

clf = MultinomialNB()
model =clf.fit(matrix_train, label)


predictions = model.predict(matrix_test)

print(predictions[:3])


df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_multinomial_unigram.csv', index=False, header=False)

submission = pd.read_csv("submission_multinomial_unigram.csv", delimiter=",", encoding="Latin-1")
submission

# 3. SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')
# SVM = svm.SVC(C=3.0, kernel='linear', degree=6, gamma='auto')
# SVM = svm.SVC(C=2.0, kernel='linear', degree=5, gamma='auto')
model = SVM.fit(matrix_train, label)

predictions = model.predict(matrix_test)
print(predictions)


df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_unigram_svm.csv', index=False, header=False)

submission = pd.read_csv("submission_unigram_svm.csv", delimiter=",", encoding="Latin-1")
submission

**2.  TFIDF**

In [0]:
vectorizer = TfidfVectorizer(max_features = 6600)
# vectorizer = TfidfVectorizer(max_features = 5000)
# vectorizer = TfidfVectorizer(max_features = 100000)

# 1. Logistic Regression

matrix_train = vectorizer.fit_transform(clean_tweet)
matrix_test = vectorizer.transform(clean_tweet_test)

lr_unigram = LogisticRegression()
lr_unigram.fit(matrix_train, label)

pred_unigram = lr_unigram.predict(matrix_test)
print(pred_unigram)

id_target = raw_data_test['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_unigram})
df_out.to_csv('submission_tfidf_test.csv', index=False, header=False) # 

submission = pd.read_csv("submission_tfidf_test.csv",
delimiter=",", encoding="Latin-1")

submission[:3] 

# 2. Multinomial Naive Bayes

clf = MultinomialNB()
model =clf.fit(matrix_train, label)


predictions = model.predict(matrix_test)

print(predictions[:3])


df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_multinomial_tfidf.csv', index=False, header=False)

submission = pd.read_csv("submission_multinomial_tfidf.csv", delimiter=",", encoding="Latin-1")
submission

# 3. SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')
# SVM = svm.SVC(C=3.0, kernel='linear', degree=6, gamma='auto')
# SVM = svm.SVC(C=2.0, kernel='linear', degree=5, gamma='auto')

model = SVM.fit(matrix_train, label)

predictions = model.predict(matrix_test)
print(predictions)


df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_tfidf_svm.csv', index=False, header=False)

submission = pd.read_csv("submission_tfidf_svm.csv", delimiter=",", encoding="Latin-1")
submission